In [1]:
import numpy as np
import logging

from geodesic import compute_shortest_paths_dijkstra_cugraph, compute_shortest_paths_dijkstra,construct_knn_graph, GPU_AVAILABLE
from dissimilarity import calculate_cs_dissimilarity_matrix
from dataset import npy_file_name_converter, read_npy_of_file, read_all_of_huaweicup

Round_idx = 1
File_idx = 1
bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num, anch_pos, H, d_geo = read_all_of_huaweicup(Round_idx,File_idx)

# 初始化日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


Processing Round 1 Case 1
Loading configuration data file
Loading input position file
Loading input CSI data of Case 1
Loading Channel CSI succeed
Channel GEO data not exist


In [3]:

# 计算所有点之间的CS距离,得到tol_samp_num*tol_samp_num大小的矩阵
# 文件路径和文件名设置
d_cs = read_npy_of_file(1, 1, 'DCS')

# 检查文件是否存在
if d_cs.any() == None : 
    # 如果文件不存在，则计算数据并保存到文件
    adp_file = npy_file_name_converter(1,1,'DCS')
    d_cs = calculate_cs_dissimilarity_matrix(H)
    np.save(adp_file, d_cs)
    print("计算并保存Dcs数据成功")

Loading ./CompetitionData1/Round1DCS1.npy


In [4]:

# 计算接地线距离
# 文件路径和文件名设置

if d_geo == None:
    # 如果文件不存在，则计算数据并保存到文件
    k = 20
    geo_file = npy_file_name_converter(1,1,'GEO')
    np.set_printoptions(precision=5, threshold=10, edgeitems=10, linewidth=200)
    print(f'构建的相异度矩阵：\n{d_cs[:10, :10]}')  # 打印前 10 行 10 列用于检查
    
    # 构建 k-近邻图
    print('开始构建 k-近邻图...')
    knn_graph_matrix = construct_knn_graph(d_cs, k)
    
    # 选择使用 CuGraph 或 NetworkX 计算最短路径长度矩阵 D_geo
    if GPU_AVAILABLE:
        print('检测到 GPU，使用 CuGraph 计算最短路径...')
        D_geo = compute_shortest_paths_dijkstra_cugraph(knn_graph_matrix)
    else:
        print('未检测到 GPU，使用 NetworkX 计算最短路径...')
        D_geo = compute_shortest_paths_dijkstra(knn_graph_matrix)
    
    # 保存 D_geo
    np.save(geo_file, D_geo)
    print('最短路径矩阵已保存。')

构建的相异度矩阵：
[[0.00000e+00 8.14148e+02 8.15650e+02 8.14144e+02 8.13561e+02 8.13490e+02 6.60232e+02 8.12416e+02 7.93362e+02 6.92680e+02]
 [8.14148e+02 0.00000e+00 7.83344e+02 6.83643e+02 7.54959e+02 7.97182e+02 8.12856e+02 7.47023e+02 7.21901e+02 8.10334e+02]
 [8.15650e+02 7.83344e+02 0.00000e+00 8.04393e+02 8.05191e+02 8.13289e+02 8.15620e+02 7.97993e+02 8.08340e+02 8.13516e+02]
 [8.14144e+02 6.83643e+02 8.04393e+02 2.40803e-05 8.04874e+02 7.79598e+02 8.13763e+02 7.11377e+02 7.17783e+02 8.14272e+02]
 [8.13561e+02 7.54959e+02 8.05191e+02 8.04874e+02 0.00000e+00 7.96701e+02 8.14681e+02 7.37783e+02 7.94918e+02 8.07468e+02]
 [8.13490e+02 7.97182e+02 8.13289e+02 7.79598e+02 7.96701e+02 0.00000e+00 8.12267e+02 6.08919e+02 7.72436e+02 8.12743e+02]
 [6.60232e+02 8.12856e+02 8.15620e+02 8.13763e+02 8.14681e+02 8.12267e+02 1.00136e-05 8.11552e+02 7.83966e+02 8.15002e+02]
 [8.12416e+02 7.47023e+02 7.97993e+02 7.11377e+02 7.37783e+02 6.08919e+02 8.11552e+02 0.00000e+00 7.43889e+02 8.08204e+02]
 [7.93

2024-07-19 23:04:40,479 - INFO - 构建的 k-近邻图，这里对k进行排序，把最小的k个邻接矩阵搬到图里，并将对角线置为0：
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


检测到 GPU，使用 CuGraph 计算最短路径...


/root/miniconda3/lib/python3.10/site-packages/cugraph/structure/symmetrize.py:92: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(
2024-07-19 23:04:42,450 - INFO - 开始计算最短路径...
计算最短路径节点数: 100%|██████████| 20000/20000 [16:07<00:00, 20.68it/s]
2024-07-19 23:20:49,565 - INFO - 计算完成，最短路径矩阵的前 10 行 10 列：
[[   0.      2339.73608 2482.6543  2267.93457 3154.5166  3232.72754 1131.91809 2917.39429 2323.21582 1640.31592]
 [2339.73608    0.      1570.17566  932.56323 2009.21497 1940.4447  2086.62354 1689.47888 1855.88013 1847.75598]
 [2482.6543  1570.17566    0.      1780.7356  2895.19971 2534.32251 2052.36182 2519.54907 2064.16382 2545.9104 ]
 [2267.93457  932.56329 1780.73572    0.      2158.09302 2047.27466 1926.2135  1978.38342 1944.37012 2054.6792 ]
 [3154.5166  2009.21484 2895.19995 2158.09302    0.      1590.91443 2747.76001 1689.52393 1861.57605 1865.7

最短路径矩阵已保存。
